# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,2.17,78,61,7.26,GS,2023-12-03
1,1,lebu,-37.6167,-73.6500,9.68,73,0,8.85,CL,2023-12-03
2,2,saint croix,17.7397,-64.7388,26.25,78,40,4.63,VI,2023-12-03
3,3,saipan,15.1355,145.7010,30.39,66,20,8.23,MP,2023-12-03
4,4,port-aux-francais,-49.3500,70.2167,5.30,90,83,17.61,TF,2023-12-03


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    color="City",
    alpha=0.5,
    frame_width=800,
    frame_height=500
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df[(city_data_df['Max Temp']<28) & (city_data_df['Max Temp']>18) & 
                                (city_data_df['Wind Speed']<4.5) & (city_data_df['Cloudiness']>=0) & (city_data_df['Cloudiness']<5)]

# Drop any rows with null values
city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
160,160,badiyah,22.4500,58.8000,25.64,40,2,0.45,OM,2023-12-03
257,257,chipinge,-20.1883,32.6236,23.38,38,1,1.16,ZW,2023-12-03
258,258,cabo san lucas,22.8909,-109.9124,22.51,66,0,1.54,MX,2023-12-03
288,288,al wajh,26.2455,36.4525,19.97,64,0,1.54,SA,2023-12-03
339,339,sur,22.5667,59.5289,25.92,51,0,2.12,OM,2023-12-03
397,397,port isabel,26.0734,-97.2086,18.02,91,0,2.06,US,2023-12-03
416,416,ain sukhna,29.6002,32.3167,18.97,68,0,2.33,EG,2023-12-03
441,441,prado,-17.3411,-39.2208,22.67,87,1,2.62,BR,2023-12-03
490,490,sultanah,24.4926,39.5857,20.25,32,0,3.60,SA,2023-12-03
550,550,caninde,-4.3589,-39.3117,24.23,81,1,1.19,BR,2023-12-03


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
160,badiyah,OM,22.4500,58.8000,40,
257,chipinge,ZW,-20.1883,32.6236,38,
258,cabo san lucas,MX,22.8909,-109.9124,66,
288,al wajh,SA,26.2455,36.4525,64,
339,sur,OM,22.5667,59.5289,51,
397,port isabel,US,26.0734,-97.2086,91,
416,ain sukhna,EG,29.6002,32.3167,68,
441,prado,BR,-17.3411,-39.2208,87,
490,sultanah,SA,24.4926,39.5857,32,
550,caninde,BR,-4.3589,-39.3117,81,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "limit":10,
    "lang":"en"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index,'Lat']
    longitude = hotel_df.loc[index,'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
badiyah - nearest hotel: Al Areen Hotel Apartments
chipinge - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
al wajh - nearest hotel: فندق رالين 2
sur - nearest hotel: Sur Hotel
port isabel - nearest hotel: Queen Isabel Inn
ain sukhna - nearest hotel: العين السخنة
prado - nearest hotel: Pousada Recanto do Prado
sultanah - nearest hotel: فندق جلنار
caninde - nearest hotel: Hotel Jardineira


,City,Country,Lat,Lng,Humidity,Hotel Name
160,badiyah,OM,22.4500,58.8000,40,Al Areen Hotel Apartments
257,chipinge,ZW,-20.1883,32.6236,38,No hotel found
258,cabo san lucas,MX,22.8909,-109.9124,66,Comfort Rooms
288,al wajh,SA,26.2455,36.4525,64,فندق رالين 2
339,sur,OM,22.5667,59.5289,51,Sur Hotel
397,port isabel,US,26.0734,-97.2086,91,Queen Isabel Inn
416,ain sukhna,EG,29.6002,32.3167,68,العين السخنة
441,prado,BR,-17.3411,-39.2208,87,Pousada Recanto do Prado
490,sultanah,SA,24.4926,39.5857,32,فندق جلنار
550,caninde,BR,-4.3589,-39.3117,81,Hotel Jardineira


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    color="City",
    alpha=0.5,
    frame_width=800,
    frame_height=500,
    hover_cols=['Hotel Name','Country']
)

# Display the map
map_plot_hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)